# Report for EE559 Homework3
Name: Peijun He&emsp;&emsp;USD ID: 2734518085

## (a) Import the data

In [128]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import hamming_loss

In [2]:
df = pd.read_csv('Frogs_MFCCs.csv')
data = df.sample(frac = 0.7, random_state = 1)

## (b) Multi-class and Multi-label problem 

### ii. Gaussian kernels
&emsp;&emsp;**From the Readme.txt, the attributes have already been normalized.**

In [117]:
X_train = data.iloc[:,0:22]
y1_train = data['Family'].values
y2_train = data['Genus'].values
y3_train = data['Species'].values

In [112]:
C = 10 ** np.arange(-2.0,4.0)
Gamma = np.append(np.arange(0.1,2.1,0.1),
                  np.arange(5,51,5))
parameters = {'C':C, 'gamma':Gamma}

# Classifier for label 'Family'
svc1 = svm.SVC()
clf1 = GridSearchCV(svc1, parameters, cv=10, n_jobs=2)
clf1.fit(X_train, y1_train)

# Classifier for label 'Genus'
svc2 = svm.SVC()
clf2 = GridSearchCV(svc2, parameters, cv=10, n_jobs=2)
clf2.fit(X_train, y2_train)

# Classifier for label 'Species'
svc3 = svm.SVC()
clf3 = GridSearchCV(svc3, parameters, cv=10, n_jobs=2)
clf3.fit(X_train, y3_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]), 'gamma': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1,
        1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  5. , 10. ,
       15. , 20. , 25. , 30. , 35. , 40. , 45. , 50. ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [190]:
print('Parameters for classifier 1:')
print(clf1.best_params_)
print('Parameters for classifier 2:')
print(clf2.best_params_)
print('Parameters for classifier 3:')
print(clf3.best_params_)

Parameters for classifier 1:
{'C': 10.0, 'gamma': 2.0}
Parameters for classifier 2:
{'C': 10.0, 'gamma': 1.1}
Parameters for classifier 3:
{'C': 10.0, 'gamma': 1.1}


In [162]:
y1_predict = clf1.predict(X_train)
y2_predict = clf2.predict(X_train)
y3_predict = clf3.predict(X_train)

In [185]:
def exact_match(x,y):
    length = len(x)
    for i in range(length):
        if x[i] != y[i]:
            return 0
    return 1

In [192]:
Exact_match = 0
Hamming_loss = 0
y_train = np.vstack((y1_train,y2_train,y3_train))
y_predict = np.vstack((y1_predict,y2_predict,y3_predict))
number = y_train.shape[1]

for i in range(number):
    Exact_match += exact_match(y_train[:,i],y_predict[:,i])
    Hamming_loss += hamming_loss(y_train[:,i],y_predict[:,i])
    
Exact_match /= number
Hamming_loss /= number
print('Exact match score:',Exact_match)
print('Hamming loss:',Hamming_loss)

Exact match score: 0.9984114376489277
Hammin loss: 0.0008604712734974847


### iii. $L_1$-penalized SVM

In [216]:
lsvc = svm.LinearSVC('l1',dual=False,C=100)
lsvc.fit(X_train,y1_train)
lsvc.score(X_train,y1_train)

0.9390389197776012

In [219]:
# Classifier for label 'Family'
Lsvc1 = svm.LinearSVC('l1',dual=False)
Lclf1 = GridSearchCV(Lsvc1, {'C':C}, cv=10, n_jobs=-1)
Lclf1 = Lclf1.fit(X_train, y1_train)

# Classifier for label 'Genus'
Lsvc2 = svm.LinearSVC('l1',dual=False)
Lclf2 = GridSearchCV(Lsvc2, {'C':C}, cv=10, n_jobs=-1)
Lclf2 = Lclf2.fit(X_train, y2_train)

# Classifier for label 'Species'
Lsvc3 = svm.LinearSVC('l1',dual=False)
Lclf3 = GridSearchCV(Lsvc3, {'C':C}, cv=10, n_jobs=-1)
Lclf3 = Lclf3.fit(X_train, y3_train)

In [220]:
print('Parameters for classifier 1:')
print(Lclf1.best_params_)
print('Parameters for classifier 2:')
print(Lclf2.best_params_)
print('Parameters for classifier 3:')
print(Lclf3.best_params_)

Parameters for classifier 1:
{'C': 10.0}
Parameters for classifier 2:
{'C': 1000.0}
Parameters for classifier 3:
{'C': 100.0}


In [223]:
y4_predict = Lclf1.predict(X_train)
y5_predict = Lclf2.predict(X_train)
y6_predict = Lclf3.predict(X_train)

In [224]:
Exact_match = 0
Hamming_loss = 0
y_train = np.vstack((y1_train,y2_train,y3_train))
y_predict = np.vstack((y4_predict,y5_predict,y6_predict))
number = y_train.shape[1]

for i in range(number):
    Exact_match += exact_match(y_train[:,i],y_predict[:,i])
    Hamming_loss += hamming_loss(y_train[:,i],y_predict[:,i])
    
Exact_match /= number
Hamming_loss /= number
print('Exact match score:',Exact_match)
print('Hamming loss:',Hamming_loss)

Exact match score: 0.9229547259729944
Hamming loss: 0.0468625893566323


### iv. Using $SMOTE$ to remedy class imbalance